In [73]:
from bs4 import BeautifulSoup as bs
import lxml
from random import choice
import random
import numpy as np
import time
import glob
import matplotlib.pyplot as plt

In [74]:
def ReadFile(data_set_number):
    
    if data_set_number==0:
        file_path = 'a280-n1395.txt'
    elif data_set_number==1:    
        file_path = 'a280-n279.txt'
    elif data_set_number==2: 
        file_path = 'a280-n2790.txt'
    elif data_set_number==3: 
        file_path = 'fnl4461-n22300.txt'
    elif data_set_number==4: 
        file_path = 'fnl4461-n4460.txt'
    elif data_set_number==5:  
        file_path = 'fnl4461-n44600.txt'
    elif data_set_number==6: 
        file_path = 'pla33810-n169045.txt'
    elif data_set_number==7: 
        file_path = 'pla33810-n33809.txt'
    elif data_set_number==8: 
         file_path = 'pla33810-n338090.txt'

    with open(file_path, 'r') as file:

        file_content = file.readlines()

    problem_info = {}
    node_coord_section = []
    items_section=[]

    for line in file_content:
        line = line.strip()
        if line.startswith('PROBLEM NAME:'):
            problem_info['PROBLEM_NAME'] = line.split(':')[-1].strip()
            
        elif line.startswith('KNAPSACK DATA TYPE:'):
            problem_info['KNAPSACK_DATA_TYPE'] = line.split(':')[-1].strip()
            
        elif line.startswith('DIMENSION:'):
            problem_info['DIMENSION'] = int(line.split(':')[-1].strip())
           
        elif line.startswith('NUMBER OF ITEMS:'):
            problem_info['NUMBER_OF_ITEMS'] = int(line.split(':')[-1].strip())
            
        elif line.startswith('CAPACITY OF KNAPSACK:'):
            problem_info['CAPACITY_OF_KNAPSACK'] = int(line.split(':')[-1].strip())

        elif line.startswith('MIN SPEED:'):
            problem_info['MIN_SPEED'] = float(line.split(':')[-1].strip())
            
        elif line.startswith('MAX SPEED:'):
            problem_info['MAX_SPEED'] = float(line.split(':')[-1].strip())
          
        elif line.startswith('RENTING RATIO:'):
            problem_info['RENTING_RATIO'] = float(line.split(':')[-1].strip())
            
        elif line.startswith('EDGE_WEIGHT_TYPE:'):
            problem_info['EDGE_WEIGHT_TYPE'] = line.split(':')[-1].strip()
            
        elif line.startswith('NODE_COORD_SECTION'):
            break

    for line in file_content[len(problem_info) + 1:]:

        if line.startswith('ITEMS SECTION'):
            break

        if line.strip():
            index, x, y = map(int, line.split())
            node_coord_section.append((index, x, y))


    for line in file_content[len(node_coord_section)+len(problem_info) +2 :]:
        if line.strip():
            index, x, y, z = map(int, line.split())
            items_section.append((index, x, y,z))
    
#     print(problem_info)#--------test
#     print(node_coord_section)#--------test
#     print(items_section)#--------test
    
    dimension = problem_info['DIMENSION']
    number_of_items= problem_info['NUMBER_OF_ITEMS']
    capacity_of_knapsack = problem_info['CAPACITY_OF_KNAPSACK']
    min_speed = problem_info['MIN_SPEED']
    max_speed = problem_info['MAX_SPEED']
    renting_ration=problem_info['RENTING_RATIO']
    
    return  dimension,\
            number_of_items,\
            capacity_of_knapsack,\
            min_speed,max_speed,\
            renting_ration,\
            node_coord_section,\
            items_section

In [75]:
dimension, number_of_items, capacity_of_knapsack, min_speed,max_speed, renting_ratio, node_coord_section, items_section = ReadFile(0)

In [76]:
print(node_coord_section[0])
print(type(node_coord_section[0]))
print(len(node_coord_section))
print(node_coord_section[0][2])
print(dimension)

(1, 288, 149)
<class 'tuple'>
280
149
280


<h3>Construct Distance Matrix</h3>

In [77]:
import math

In [78]:
D = []
for i, source in enumerate(node_coord_section):
    source_x = source[1]
    source_y = source[2]
    row = []
    j = 0
    while j < len(node_coord_section):
        if i==j:
            row.append(0)
        else:
            dest_x = node_coord_section[j][1]
            dest_y = node_coord_section[j][2]
            dist = math.sqrt((dest_x - source_x)**2 + (dest_y - source_y)**2)
            row.append(int(dist))
        j += 1
    D.append(row)

In [62]:
print(D)

[[0, 20, 24, 32, 32, 42, 55, 63, 61, 70, 78, 86, 94, 101, 92, 100, 116, 124, 132, 140, 148, 141, 125, 117, 133, 149, 157, 165, 172, 184, 184, 185, 198, 208, 224, 224, 232, 232, 232, 232, 232, 232, 233, 249, 248, 248, 248, 248, 248, 248, 256, 256, 256, 256, 256, 256, 257, 258, 250, 234, 236, 245, 252, 261, 262, 270, 276, 274, 282, 284, 286, 288, 290, 292, 294, 287, 297, 300, 283, 277, 274, 272, 269, 267, 264, 256, 259, 262, 266, 266, 269, 272, 284, 291, 298, 302, 295, 288, 280, 270, 266, 263, 259, 255, 248, 241, 238, 252, 259, 249, 246, 254, 251, 244, 241, 248, 239, 237, 191, 189, 187, 186, 185, 184, 184, 172, 164, 156, 156, 148, 140, 132, 124, 118, 120, 122, 125, 128, 132, 125, 129, 134, 139, 144, 150, 145, 140, 136, 141, 153, 176, 173, 168, 165, 167, 170, 193, 196, 199, 202, 209, 213, 217, 221, 226, 240, 250, 257, 257, 249, 242, 242, 234, 223, 205, 182, 179, 170, 170, 179, 186, 190, 194, 198, 203, 201, 208, 218, 212, 206, 197, 188, 181, 172, 167, 161, 166, 161, 155, 149, 155, 147, 135

<h3>Item Matrix</h3>
Help locate which items are in which city

In [79]:
print(len(items_section))

1395


In [109]:
#Get values and weights of items
value = []
weight = []
for item in items_section:
    value.append(item[1])
    weight.append(item[2])
print(len(value))
print(len(weight))

1395
1395


In [110]:
item_location = []
# for i, item in enumerate(items_section):
#     l = [j[3]-1 for j in item if ]
i = 0
while i < len(node_coord_section):
    row = [item[0]-1 for j,item in enumerate(items_section) if item[3]-1 == i]
    item_location.append(row)
    i += 1


In [125]:
# print(len(item_location))
# print(item_location)

<h3>Generate Random Solution for tsp or kp based on the flag</h3>

In [28]:
def GenerateRandomSolution(n, isTSP):
    solution = []
    if isTSP:
        while len(solution) < n:
            g = np.random.randint(n)
            #As TSP is permutation problem only one instance of a city is allowed the below check is implemented
            if g not in solution:
                solution.append(g)
    else:
        while len(solution) < n:
            g = np.random.randint(0,2)
            #As packing plan is 0 for item not picked up and 1 for item picked up create list of 0's and 1's of size n
            solution.append(g)
    return solution

In [82]:
tsp_population = GenerateRandomSolution(dimension, True)
kp_population = GenerateRandomSolution(number_of_items, False)

<h3>Generate a population of "size" number of random solutions </h3>

In [37]:
def GeneratePopulation(size, num_cities, num_items):
    tsp_pop = []
    kp_pop = []
    for i in range(size):
        tsp_pop.append(GenerateRandomSolution(num_cities,True))
        kp_pop.append(GenerateRandomSolution(num_items,False))
    return tsp_pop,kp_pop

In [83]:
tsp_population, kp_population = GeneratePopulation(500,dimension,number_of_items)

<h3>Calculate total profit of a packing plan</h3>

In [126]:
def eval_profit(packing_plan, value_list):
    picked_values = np.multiply(packing_plan,value_list)
#     print(picked_values)
    profit = sum(picked_values)
    return profit

In [127]:
eval_profit(kp_population[0],value)

336501

<h3>Calculate the speed based on the weight</h3>

In [85]:
def eval_speed(w):
    if w <= capacity_of_knapsack:
        Speed = (max_speed - (w*(max_speed-min_speed)/capacity_of_knapsack))
    else:
        Speed = min_speed
    return Speed

In [50]:
eval_speed(100000)

0.858714933831494

<h3>Calculate the time for journey</h3>
Affected by the items picked up

In [128]:
def eval_time(tour, packing_plan, item_location, item_weight):
    total_weight = 0
    total_time = 0
    for i,city in enumerate(tour):
        current_weight = []
        if i < dimension-1:
            
            items = item_location[i]
#             print(items)
            if items:
#                 for j in items:
#                     print(packing_plan[j])
#                     print(item_weight[j])
#                     current_weight.append(packing_plan[j]*item_weight[j])
                current_weight = [packing_plan[j]*item_weight[j] for j in items]
                total_weight += sum(list(current_weight))
            
            speed = eval_speed(total_weight)
            total_time += D[i][i+1]/speed
        else:
#             current_weight = 0
            items = item_location[i]
            if items:
                current_weight = [packing_plan[j]*item_weight[j] for j in items]
                total_weight += sum(list(current_weight))
            
            speed = eval_speed(total_weight)
            total_time += D[i][0]/speed
    return total_time

In [64]:
eval_time(tsp_population[0], kp_population[0], item_location, weight)

9387.768041331798

In [87]:
def eval_fitness(profit, time):
    fitness = profit - (renting_ratio*time)
    return fitness

In [67]:
eval_fitness(352813,9387.768041331798)

-329677.73660482175

<h6>Evaluate Fitness of entire population</h6>

In [94]:
def eval_population_fitness(tsp_pop, kp_pop):
    pop_fitness = []
    for tour,packing_plan in zip(tsp_pop,kp_pop):
        time = eval_time(tour, packing_plan, item_location, weight)
        profit = eval_profit(packing_plan, value)
        fitness = eval_fitness(profit, time)
        pop_fitness.append(fitness)
    return pop_fitness

<h6>Crossover functions</h6>
Crossover with fix for tsp
Normal crossover for kp

In [89]:
def CrossoverWithFix(a,b):
    #Choose a random point for crossover
    p = np.random.randint(1,len(a))
    #Split the chromosomes into 2 sections based on the crossover point
    a_left = a[:p]
    a_right = a[p:]
    b_left = b[:p]
    b_right = b[p:]
    #Form the children by combining the left section of one parent with the right section of the other parent
    a_new = a_left + b_right
    b_new = b_left + a_right
    #find missing and conflicting elements of a_new
    a_missing = [i for i in a if i not in a_new]
    a_conflict = [i for i in a_left if i in b_right]
    #fix a_new by replacing the conflicting elements with the missing elements
    for element in a_new:
        if element in a_conflict:
            copy = element
            a_new[a_new.index(element)] = choice([i for i in a_missing if i not in a_new])
            a_conflict.remove(copy)

    #find missing and conflicting elements of b_new
    b_missing = [i for i in b if i not in b_new]
    b_conflict = [i for i in b_left if i in a_right]
    #fix b_new by replacing the conflicting elements with the missing elements
    for element in b_new:
        if element in b_conflict:
            copy = element
            b_new[b_new.index(element)] = choice([i for i in b_missing if i not in b_new])
            b_conflict.remove(copy)
    
    return a_new,b_new

In [90]:
def Crossover(a,b):
    #Choose a random point for crossover
    p = np.random.randint(1,len(a))
    #Split the chromosomes into 2 sections based on the crossover point
    a_left = a[:p]
    a_right = a[p:]
    b_left = b[:p]
    b_right = b[p:]
    #Form the children by combining the left section of one parent with the right section of the other parent
    a_new = a_left + b_right
    b_new = b_left + a_right
    
    return a_new,b_new

<h6>Selection Function</h6>

In [96]:
def RankBasedSelection(tsp_pop,kp_pop,pop_fitness):
    #Combine the population and its fitness
    fitness_population = sorted(zip(pop_fitness, tsp_pop, kp_pop))
    #Sort the combined population and fitness based on the fitness values
    sorted_tsp_pop = [y for x,y,z in fitness_population]
    sorted_kp_pop = [z for x,y,z in fitness_population]
    sorted_pop_fitness = [x for x,y,z in fitness_population]
    rank = []
    #Assign ranks to the solutions in the population
    for i in range(len(fitness_population)):
        rank.append(i+1)
    rank_total = sum(rank)
    probability = []
    #Find the probabilities of choosing a solution from the population based on its rank
    for r in rank:
        probability.append(r/rank_total)
    indeces = np.arange(len(fitness_population))
    #Choose 2 parents randomly based on the rank based probabilities calculated
    choice_1, choice_2 = np.random.choice(indeces,2,replace=True,p=probability)
#     parent_1, parent_2 = sorted_population[choice_1], sorted_population[choice_2]
#     return parent_1,parent_2
    return choice_1,choice_2

<h3>Replacement Funtion</h3>

In [92]:
def ReplaceWeakest(tsp_pop, kp_pop, pop_fitness, child_tsp, child_kp, child_fitness):
    #Find the worst fitness in the population
    worst_fitness = min(pop_fitness)
    #Replace the worst solution if the new candidate solution is better or the same
    if child_fitness >= worst_fitness:
        list_index = pop_fitness.index(worst_fitness)
        del pop_fitness[list_index]
        del tsp_pop[list_index]
        del kp_pop[list_index]
        tsp_pop.append(child_tsp)
        kp_pop.append(child_kp)
        pop_fitness.append(child_fitness)
    return tsp_pop, kp_pop, pop_fitness

<h3>10000 runs of GA - Rank selection, Crossover, Replace Weakest</h3>

In [137]:
tsp_population, kp_population = GeneratePopulation(500,dimension,number_of_items)
print(type(kp_population))
population_fitness = eval_population_fitness(tsp_population, kp_population)
avg_fitness = sum(population_fitness)/dimension
print("Average Fitness - ",avg_fitness)
for i in range(10000):
    p1, p2 = RankBasedSelection(tsp_population,kp_population,population_fitness)
    tsp_p1 = tsp_population[p1]
    tsp_p2 = tsp_population[p2]
#     print("TSP P1 - ",tsp_p1)
#     print("TSP P2 - ",tsp_p2)
    kp_p1 = kp_population[p1]
    kp_p2 = kp_population[p2]
#     print("KP P1 - ",kp_p1)
#     print("KP P2 - ",kp_p2)
    tsp_child1, tsp_child2 = CrossoverWithFix(tsp_p1,tsp_p2)
    kp_child1, kp_child2 = Crossover(kp_p1,kp_p2)
#     print("TSP child - ",tsp_child)
#     print(len(tsp_child1))
#     print(type(tsp_child1))
#     print("KP child - ",kp_child)
#     print(len(kp_child1))
#     print(type(kp_child1))
    child1_time = eval_time(tsp_child1, kp_child1, item_location, weight)
    child1_profit = eval_profit(kp_child1, value)
    child1_fitness = eval_fitness(child1_profit, child1_time)
    tsp_population, kp_population, population_fitness = ReplaceWeakest(tsp_population, kp_population, population_fitness, tsp_child1, kp_child1, child1_fitness)
    
    child2_time = eval_time(tsp_child2, kp_child2, item_location, weight)
    child2_profit = eval_profit(kp_child2, value)
    child2_fitness = eval_fitness(child2_profit, child2_time)
    tsp_population, kp_population, population_fitness = ReplaceWeakest(tsp_population, kp_population, population_fitness, tsp_child2, kp_child2, child2_fitness)
#     ReplaceWeakest(tsp_pop, kp_pop, pop_fitness, child_tsp, child_kp, child_fitness)
    avg_fitness = sum(population_fitness)/dimension
    print(avg_fitness)

<class 'list'>
Average Fitness -  -553076.7975917738
-552419.042828265
-551705.9498899946
-550924.6212883516
-550392.2133373775
-550210.9837788786
-549952.9489087813
-549732.6786300406
-549584.0081585869
-549216.1048796641
-548952.6265800492
-548835.1565528137
-548579.8066028323
-548087.0518625963
-547738.7137546135
-547422.6092329848
-547173.3204068742
-546762.3755784631
-546436.4935763207
-546170.4596264032
-545945.6471649221
-545411.1826597995
-545105.3079872736
-544716.1315876922
-544464.6463495163
-544141.6378888686
-543911.5477840889
-543429.8665480997
-542974.4868570338
-542324.0281706956
-542028.7104481829
-541756.5670102427
-541578.3764317898
-541091.5063079594
-540889.6604293492
-540714.8550976596
-540405.33154536
-539951.979071416
-539647.7367073429
-539408.8432164731
-539063.3574881535
-538956.8869379023
-538821.5856745292
-538542.9985578454
-538305.6895544919
-538174.4642535922
-537703.7026056537
-537588.0122376336
-537253.0809905089
-537139.946391799
-536726.0297290867
-5

-461080.0020440515
-460807.4892314185
-460634.9037295213
-460445.9780108629
-460289.8059267026
-460125.4208040135
-459894.1292725614
-459860.1422320348
-459504.647128016
-459209.47436157445
-458933.54826121026
-458712.3098103702
-458551.43068456504
-458447.44928067666
-458401.80548471044
-458345.4134072914
-458216.41834662296
-458093.1745037384
-457914.8700847986
-457831.8951688064
-457752.3927069123
-457508.5656570304
-457352.60702832596
-457139.25467851013
-457048.1872036417
-456909.06730087387
-456803.03725282755
-456731.82765923213
-456703.0650256107
-456622.68300712045
-456441.0428841001
-456315.77397254936
-456122.07652533916
-455995.8755285279
-455684.71308113233
-455542.36993188754
-455287.33670960646
-455176.11006256816
-454996.8794521615
-454917.75232185895
-454646.69001162593
-454425.8312656833
-454289.8994269918
-454110.8494300141
-453924.2173794306
-453734.6080198813
-453559.6137470796
-453436.1430941997
-453366.43025575596
-453206.32159001543
-452981.4137080739
-452779.20

-390510.4900086911
-390225.0877152264
-390072.5872812822
-389817.7865246592
-389638.9329709021
-389492.12957576907
-389344.7355600581
-389072.0963475341
-388851.66045834945
-388599.31363394205
-388399.0794533006
-388262.7639100918
-388226.9347195145
-388033.9454284905
-387916.8659094825
-387802.69077073596
-387709.2782624243
-387519.9872131446
-387396.6711115147
-387130.92405445565
-386944.1957765495
-386758.33941092464
-386563.0835651038
-386287.92339467775
-386246.04016416
-386093.4664845651
-385948.5547135792
-385772.5551490975
-385659.0191325131
-385477.3788369744
-385152.68037459726
-384990.1607640493
-384852.0074948728
-384740.85913189047
-384605.6420490687
-384486.0157707443
-384324.3563093646
-384191.5091669651
-384133.8758874703
-383994.29124811885
-383732.2798178447
-383544.8176068935
-383473.4316272429
-383206.7597443233
-383047.27513772907
-382869.2763823781
-382669.7532343735
-382564.00747030013
-382470.84183111065
-382376.9221280241
-382249.19762191945
-382072.0770308372


-325057.4691332905
-324874.0372975993
-324812.27620590525
-324630.8456507096
-324560.9688196258
-324271.2808509853
-324144.66589798225
-324012.39099568763
-323876.7055064174
-323718.1695692258
-323636.63152719947
-323358.27441148297
-323190.5938103586
-323127.11963344493
-322928.4461616919
-322807.5180950261
-322757.56160990195
-322668.27673945343
-322624.37019008194
-322581.88104830944
-322471.29806104937
-322254.01608456083
-322138.01177848416
-321970.47193836595
-321928.77603829995
-321680.4905249806
-321583.8144146536
-321217.17513632955
-321165.9816977706
-321001.840115236
-320886.64895824034
-320789.8395547472
-320690.6111447007
-320631.99993865
-320558.675106511
-320467.962530006
-320251.5736699463
-320212.84007013746
-320099.21400098654
-319960.3304592445
-319848.6525721757
-319707.74395566946
-319607.398777988
-319541.74138089665
-319453.0464318121
-319310.5116316233
-319199.4127601328
-319080.9834875672
-318946.22463835374
-318770.45544043905
-318639.66269241023
-318597.64728

-275892.71776983794
-275768.31451397715
-275621.8113107914
-275577.6042980025
-275482.2093345507
-275403.68860879407
-275322.4636963988
-275230.29916550766
-275156.9842574664
-275031.46526135417
-274971.9158966353
-274889.8924849399
-274769.8660595631
-274734.5736302907
-274659.69451795757
-274509.5684894138
-274436.7264035641
-274322.58064736216
-274286.15254125156
-274208.28394101746
-274164.56711460324
-274078.8778347155
-274037.10340116016
-273939.35513778246
-273804.82619198324
-273689.5787990058
-273620.3545092943
-273535.9683054038
-273433.7488759419
-273340.5807037961
-273137.26992824924
-273045.76368270617
-272976.8267526344
-272891.5253620072
-272724.912253856
-272678.82901065605
-272548.671069003
-272422.0204448958
-272337.97039140173
-272211.1832924835
-272150.64187791356
-272074.6358132448
-271946.79933485383
-271849.35456539784
-271676.3424216164
-271569.12125545315
-271447.71910082083
-271356.1222423101
-271268.11588257493
-271150.35965703096
-271044.74654057337
-270944.

...

-92857.86366733906
-92856.46308696803
-92847.95326100702
-92845.7605797196
-92838.3200874856
-92832.16888118025
-92829.94854988632
-92822.86831076648
-92818.22232193947
-92816.64265956832
-92809.68841945611
-92806.03886342666
-92795.35424092515
-92791.50082029343
-92786.89253820754
-92775.37527124301
-92773.29709058437
-92772.70448674
-92760.76942596129
-92755.02357293751
-92752.05485267402
-92751.16944684289
-92747.91720432913
-92747.24436079191
-92745.61938411974
-92737.90639754735
-92734.32825034812
-92728.31514816395
-92720.60271595517
-92710.5555784728
-92702.18879131875
-92696.5831841368
-92688.65842870685
-92682.19679179696
-92676.01712062476
-92671.7493706384
-92671.46697007526
-92665.66618291574
-92661.81519016561
-92659.49779635188
-92654.91545342738
-92647.91738313784
-92642.688529222
-92637.53592104407
-92630.73755328606
-92627.37300976887
-92613.14687295638
-92608.37661216958
-92602.73882579384
-92598.01828650336
-92592.86927317681
-92591.73153015827
-92583.25875843542
-92

-90102.32947724641
-90099.01831146269
-90091.0560655737
-90084.79505228178
-90077.42338818435
-90074.82425321674
-90069.37814804874
-90061.38412703389
-90055.33973567645
-90053.8878792019
-90049.75433303662
-90045.20548020962
-90041.43854742096
-90027.44905298957
-90021.91142287765
-90009.68983426197
-90000.6215786065
-89992.88162314241
-89987.7760706412
-89983.97408572557
-89972.99103680138
-89968.95683647307
-89962.6093433934
-89957.42705593436
-89951.49514937968
-89947.65881374973
-89944.16033134989
-89940.94039218206
-89937.26996847648
-89929.01963860623
-89918.79557351858
-89916.72389043964
-89911.44144930289
-89904.65243216539
-89902.02638393485
-89892.30703932425
-89887.87622516333
-89883.19400391274
-89882.00387897319
-89875.86978316538
-89867.06238147806
-89858.52851776379
-89853.46158496158
-89844.49313600159
-89834.16214801707
-89826.5785904978
-89823.6733026059
-89818.06234593775
-89811.20508892187
-89808.91250035097
-89799.84993142508
-89791.54759952729
-89781.30147283054


-87503.61338966757
-87497.96595110866
-87490.89258821006
-87490.17311500774
-87484.932550797
-87481.17917876487
-87475.60632093491
-87465.05178075888
-87458.33892526265
-87457.34286231967
-87454.39220398005
-87447.0387774126
-87444.19323618362
-87436.81017058053
-87431.21627594886
-87429.76000587312
-87422.5951063076
-87421.73179157896
-87415.03861034197
-87413.6122890535
-87410.1127000632
-87406.04556154259
-87403.53130654313
-87392.04074113305
-87389.44667181194
-87381.4894547491
-87377.16403505008
-87374.68313766178
-87372.44404869054
-87366.01753774831
-87360.62620478883
-87355.79561731928
-87353.86551564973
-87347.48797876488
-87341.73432768397
-87335.7121758088
-87330.28843163967
-87325.27812397471
-87320.65067512316
-87308.29106520647
-87306.91790658444
-87300.66552499798
-87293.38167825753
-87283.76561799465
-87282.58171056974
-87279.48860134596
-87275.78434325938
-87266.46497775117
-87261.81514433988
-87248.76463885553
-87239.80358905229
-87238.48623679331
-87231.54234255485
-

-85417.30075966823
-85414.18653664121
-85410.78451174394
-85404.27252758741
-85401.0340677559
-85393.64091838867
-85391.01275244744
-85386.78932885166
-85385.42439713496
-85380.34359728007
-85374.30584321693
-85370.66964243962
-85367.3100798227
-85362.73617465363
-85359.54162202326
-85356.96300833963
-85353.94840960664
-85348.14466213838
-85340.93636301764
-85334.68240435563
-85333.07311261843
-85330.28400514438
-85323.30079901085
-85313.16860308085
-85308.92511452736
-85303.86668712119
-85300.41392817465
-85298.45226557668
-85294.53109895706
-85290.25115386497
-85288.8679118106
-85288.52818734544
-85282.50353631208
-85281.53152379107
-85273.33905486831
-85272.70857700065
-85270.96836331296
-85265.89892375314
-85258.21727192066
-85254.43623636835
-85250.32529712554
-85248.51346587898
-85243.52970652957
-85238.56784929303
-85233.31345758055
-85232.62946789071
-85226.22256749822
-85224.54177521712
-85219.72932410828
-85219.31689540816
-85215.92835771156
-85211.28397485659
-85206.77898337

-83668.72313656386
-83661.828942999
-83656.7675155045
-83652.7965898863
-83648.90842887555
-83645.54769585257
-83642.15620021142
-83634.62584192112
-83631.17780922251
-83629.9989749537
-83625.42455058977
-83622.73209081421
-83611.20284536465
-83606.91600455026
-83598.61446216906
-83594.88886676035
-83581.93677007937
-83578.16747861577
-83576.5084880699
-83571.07902794467
-83568.41951998453
-83567.07624698612
-83562.3760689369
-83560.1991534742
-83551.18734261773
-83546.77214724239
-83540.82460411708
-83538.26332651426
-83537.02732098452
-83532.87224626273
-83528.41465417316
-83528.04210192135
-83527.16325721126
-83519.56132435748
-83515.28757489976
-83513.7308536507
-83511.7298988549
-83508.43855699946
-83504.16660743074
-83503.03745216521
-83497.2639754144
-83488.42610691638
-83485.86955097498
-83482.28257587376
-83480.2342238022
-83476.82297539388
-83473.29122012744
-83471.2882336936
-83467.51870695304
-83465.0627800071
-83461.49111871966
-83460.79220934286
-83449.71437542464
-83448.